# 8. Deep Learning wirh PyTorch

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# 8.1 Getting Started with PyTorch

<h3> 8.1.1 Autograd : 자동 미분

In [2]:
# 먼저 tensor 타입의 변수를 선언하고, 계산 그래프를 생성

x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

y = w * x + b

# backward()함수로 계산 그래프로부터 미분값 계산
y.backward()

# 출력
print(x.grad)
print(w.grad)
print(b.grad)

tensor(2.)
tensor(1.)
tensor(1.)


<h3> 8.1.2 Loading data from numpy : ndarray -> torch tensor 변환

---



In [3]:
import numpy as np

# numpy array 생성
x = np.array([[1,2], [3,4]])

# numpy array -> torch tensor
y = torch.from_numpy(x)

# torch tensor -> numpy array
z = y.numpy()

<h3> 8.1.3 DataLoader : 모델에 데이터 공급 </h3>   
sklearn.dataset과 같이 자주 쓰는 데이터를 내장하고 있음

In [4]:
# CIFAR-10 데이터셋을 다운로드 및 메모리에 로드
train_dataset = torchvision.datasets.CIFAR10(root='../../data/', 
                                             train = True,
                                             transform = transforms.ToTensor(),
                                             download=True)

# 데이터셋의 첫 번째 instance로 접근해보기
image, label = train_dataset[0]
print(image.size())
print(label)

# 데이터세스로부터 DataLoader 생성
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64,
                                           shuffle=True)

# Iterator을 사용하여 미니배치를 구현할 수도 있음
#data_iter = iter(train_loader)
#images, lables = data_iter.next()

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../../data/cifar-10-python.tar.gz to ../../data/
torch.Size([3, 32, 32])
6


<h3> 8.1.4 Loading a Pretrained Model : 모델 파일 형태로 주고받기</h3>

In [5]:
# 이전에 학습된 ResNet-18 다운로드 및 불러오기
resnet = torchvision.models.resnet18(pretrained=True)

# ResNet-18의 가장 마지막 레이어만 추가 학습
for param in resnet.parameters():
  param.requires_grad = False
resnet.fc = nn.Linear(resnet.fc.in_features, 100)

# Forward pass를 통해 학습된 모델을 예측 작업에 바로 적용해볼 수 있음
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print(outputs.size())

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

torch.Size([64, 100])


# 8.2 Logistic Regression (PyTorch)

In [6]:
# 1. 실습에 사용할 데이터 불러와 DataLoader 구상

input_size = 784
num_classes = 10
batch_size = 100

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data',
                                           train=True,
                                           transform = transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data',
                                           train=False,
                                           download=True)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw



In [7]:
# Pytorchfh logistic regression 모델을 정의하는 방법

# Hyper-parameter setting: 실험에 사용할 하이퍼파라미터들 설정
learning_rate = 0.001

# Model definitiom : 하나의 linear 레이어로 구상된 네트워크 만들기
model = nn.Linear(input_size, num_classes)

# Training_parameter setting : 모델이 사용할 loss func와 ptimizer 지정
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [8]:
# 위 모델들을 위한 학습 및 예측 함수

# Train the model
def train_logreg(train_loader, num_epoches):
  total_step = len(train_loader)
  for epoch in range(num_epoches):
    for i, (images, labels) in enumerate(train_loader):
      images = images.reshape(-1, 28*28)

      outputs = model(images)
      loss = criterion(outputs, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if (i+1) % 300 == 0:
        print('Epoch [{}/{}], Step [{}{}], Loss: {:4f}'
        .format(epoch+1, num_epoches, i+1, total_step, loss.item()))

def test_logreg(model, test_loader):
  with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
      images = images.reshape(-1, 28*28)
      outputs = model(images)
      _, predicted = torch.max(outputs.data,1)
      total += labels.size(0)
      correct += (predicted == labels).sum()

  print('Accuracy of the model on the 10000 test images: {} %' ,format(100 * correct / total))

In [9]:
train_logreg(train_loader, num_epoches=10)
test_logreg(model, test_loader)

Epoch [1/10], Step [300600], Loss: 2.063842
Epoch [1/10], Step [600600], Loss: 1.831737
Epoch [2/10], Step [300600], Loss: 1.595085
Epoch [2/10], Step [600600], Loss: 1.463220
Epoch [3/10], Step [300600], Loss: 1.343986
Epoch [3/10], Step [600600], Loss: 1.201803
Epoch [4/10], Step [300600], Loss: 1.234993
Epoch [4/10], Step [600600], Loss: 1.131652
Epoch [5/10], Step [300600], Loss: 0.991573
Epoch [5/10], Step [600600], Loss: 0.975365
Epoch [6/10], Step [300600], Loss: 1.086007
Epoch [6/10], Step [600600], Loss: 0.938147
Epoch [7/10], Step [300600], Loss: 0.910210
Epoch [7/10], Step [600600], Loss: 0.858466
Epoch [8/10], Step [300600], Loss: 0.922782
Epoch [8/10], Step [600600], Loss: 0.943028
Epoch [9/10], Step [300600], Loss: 0.873868
Epoch [9/10], Step [600600], Loss: 0.792670
Epoch [10/10], Step [300600], Loss: 0.788196
Epoch [10/10], Step [600600], Loss: 0.897379
Accuracy of the model on the 10000 test images: {} % 84.42833709716797


# 8.3 Feedforward Neural Networks

In [10]:
# 하이퍼파라메터 설정

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

hidden_size = 500
learning_rate = 0.001

In [11]:
# FFNet 모델 정의

class FFNet(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(FFNet, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [12]:
# 학습 및 예측 함수 정의

def train_ffnet(model, train_loader, num_epochs):
  total_step = len(train_loader)
  for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
      images = images.reshape(-1, 28*28).to(device)
      labels = labels.to(device)

      outputs = model(images)
      loss = criterion(outputs, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if (i+1) % 300 == 0:
        print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
          .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        
def test_ffnet(model, test_loader):
  with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
      images = images.reshape(-1, 28*28).to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs.data,1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  print('Accuracy of the network on the 10000 test images: {} %' ,format(100 * correct / total))

In [13]:
# MNIST 데이터에 적용

model = FFNet(input_size, hidden_size, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_ffnet(model, train_loader, num_epochs=10)
test_ffnet(model, test_loader)


Epoch [1/10], Step [300/600], Loss: 0.2313
Epoch [1/10], Step [600/600], Loss: 0.1030
Epoch [2/10], Step [300/600], Loss: 0.1086
Epoch [2/10], Step [600/600], Loss: 0.1717
Epoch [3/10], Step [300/600], Loss: 0.1115
Epoch [3/10], Step [600/600], Loss: 0.1095
Epoch [4/10], Step [300/600], Loss: 0.0935
Epoch [4/10], Step [600/600], Loss: 0.0432
Epoch [5/10], Step [300/600], Loss: 0.1516
Epoch [5/10], Step [600/600], Loss: 0.0351
Epoch [6/10], Step [300/600], Loss: 0.0580
Epoch [6/10], Step [600/600], Loss: 0.0253
Epoch [7/10], Step [300/600], Loss: 0.0290
Epoch [7/10], Step [600/600], Loss: 0.0185
Epoch [8/10], Step [300/600], Loss: 0.0074
Epoch [8/10], Step [600/600], Loss: 0.0269
Epoch [9/10], Step [300/600], Loss: 0.0036
Epoch [9/10], Step [600/600], Loss: 0.0124
Epoch [10/10], Step [300/600], Loss: 0.0022
Epoch [10/10], Step [600/600], Loss: 0.0396
Accuracy of the network on the 10000 test images: {} % 99.62666666666667


# 8.4 Convolutional Neural Networks

In [14]:
# 하이퍼파라메터 설정

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

learning_rate = 0.001

In [15]:
# CNN 모델 정의

class ConvNet(nn.Module):
  def __init__(self, num_classes=10):
    super(ConvNet, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.fc = nn.Linear(7*7*32, num_classes)
  
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.reshape(out.size(0), -1)
    out = self.fc(out)
    return out

In [16]:
# 학습 및 예측 함수 정의

def train_convnet(model, train_loader, num_epochs):
  total_step = len(train_loader)
  for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
      images = images.to(device)
      labels = labels.to(device)

      outputs = model(images)
      loss = criterion(outputs, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if (i+1) % 300 == 0:
        print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
          .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        
def test_convnet(model, test_loader):
  model.eval()
  with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs.data,1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  print('Accuracy of the model on the 10000 test images: {} %' ,format(100 * correct / total))

In [27]:
"""
This is the c3d implementation with batch norm.
References
----------
[1] Tran, Du, et al. "Learning spatiotemporal features with 3d convolutional networks." 
Proceedings of the IEEE international conference on computer vision. 2015.
"""

import math
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Variable
from functools import partial


class C3D(nn.Module):
    def __init__(self,
                 sample_size,
                 sample_duration,
                 num_classes=600):
      
      # 3개의 convoltution layer + 2개의 sub-sampling layer, 1개의 fully connected layer
       #input - C1 - S2 - C3 - S4 - C5 - F6 - output

       # sub-sampling layer == pooling layer

        super(C3D, self).__init__()

        # input -> C1 : 첫 번째 convolution layer
        self.group1 = nn.Sequential(
            nn.Conv3d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(1, 2, 2)))
        
        # C1 -> S2 : 
        self.group2 = nn.Sequential(
            nn.Conv3d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm3d(128),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2)))
        self.group3 = nn.Sequential(
            nn.Conv3d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm3d(256),
            nn.ReLU(),
            nn.Conv3d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm3d(256),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2)))
        self.group4 = nn.Sequential(
            nn.Conv3d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm3d(512),
            nn.ReLU(),
            nn.Conv3d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm3d(512),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2)))
        self.group5 = nn.Sequential(
            nn.Conv3d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm3d(512),
            nn.ReLU(),
            nn.Conv3d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm3d(512),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(2, 2, 2), padding=(0, 1, 1)))

        last_duration = int(math.floor(sample_duration / 16))
        last_size = int(math.ceil(sample_size / 32))
        self.fc1 = nn.Sequential(
            nn.Linear((512 * last_duration * last_size * last_size) , 4096),
            nn.ReLU(),
            nn.Dropout(0.5))
        self.fc2 = nn.Sequential(
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5))
        self.fc = nn.Sequential(
            nn.Linear(4096, num_classes))         

        

    def forward(self, x):
        out = self.group1(x)
        out = self.group2(out)
        out = self.group3(out)
        out = self.group4(out)
        out = self.group5(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc(out)
        return out


def get_fine_tuning_parameters(model, ft_portion):
    if ft_portion == "complete":
        return model.parameters()

    elif ft_portion == "last_layer":
        ft_module_names = []
        ft_module_names.append('fc')

        parameters = []
        for k, v in model.named_parameters():
            for ft_module in ft_module_names:
                if ft_module in k:
                    parameters.append({'params': v})
                    break
            else:
                parameters.append({'params': v, 'lr': 0.0})
        return parameters

    else:
        raise ValueError("Unsupported ft_portion: 'complete' or 'last_layer' expected")


def get_model(**kwargs):
    """
    Returns the model.
    """
    model = C3D(**kwargs)
    return model


if __name__ == '__main__':
    model = get_model(sample_size = 112, sample_duration = 16, num_classes=600)
    model = model.cuda()
    model = nn.DataParallel(model, device_ids=None)
    print(model)

    input_var = Variable(torch.randn(8, 3, 16, 112, 112))
    output = model(input_var)
    print(output.shape)

RuntimeError: ignored

In [26]:
# MNIST 데이터에 적용

model = ConvNet(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_convnet(model, train_loader, num_epochs=10)
test_convnet(model, test_loader)

KeyboardInterrupt: ignored

# 8.5 Playing with Pre-trained Network

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
!pip install pretrainedmodels
!python ./drive/MyDrive/imagenet_install/setup.py install

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 4.1 MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=12c68818144d08ba3e597119d2b86384a239a9f1df5b33b7b28770e9d9b5824a
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built pretrainedmodels
running install
running bdist_egg
running egg_info
creating pretrainedmodels.egg-info
writing pretrainedmodels.egg-info/PKG-INFO
writing dependency_links to pretrainedmodels.egg-info/dependency_links.txt
writing requirements to pretrainedmodels.egg-info/requires.txt
writing top-level names to pretrainedmodels.egg-info/top_level.txt
writing manifest file 'pretrainedmodels.egg-info/SOURCES.txt'
reading manifest file 'pretrainedmodels.egg-info/SOURCES.txt'
writing manifest file 'pretrainedmodels.egg-info/SOURCES.txt'
installing l

In [20]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [24]:
import torch
import pretrainedmodels
import pretrainedmodels.utils as utils

model_name = 'nasnetalarge'
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
model.eval()

load_img = utils.LoadImage()

tf_img = utils.TransformImage(model)

AttributeError: ignored

In [ ]:
torch.save(model, 'drive/MyDrive/imagenet_nasnetalarge.pth')

In [22]:
import csv
name_file = 'drive/MyDrive/imagenet_install/class_names.csv'

imagenet_class = {}
file_in = csv.reader(open(name_file))
for row in file_in:
  imagenet_class[int(row[0])] = row[1]

In [25]:
# 예측코드 1
load_img = utils.LoadImage()

try:
  model
except NameError:
  import pretrainedmodels.utils as utils
  model = torch.load('drive/MyDrive/imagenet_nasnetalarge.pth')
  laod_img = utils.LoadImage()
  tf_img = utils.LoadImage()

img_file = './drive/MyDrive/002.jpeg'

input_img = load_img(img_file)
input_tensor = tf_img(input_img)
input_tensor = input_tensor.unsqueeze(0)
input = torch.autograd.Variable(input_tensor, requires_grad=False)

output_logits = model(input)

print("{} is [{}: {}]".format(img_file, output_logits.argmax(),
                              imagenet_class[int(output_logits.argmax())]))

NameError: ignored

<h1> test </h1>

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Variable
from functools import partial


class C3D(nn.Module):
    def __init__(self,
                 sample_size,
                 sample_duration,
                 num_classes=600):

        super(C3D, self).__init__()
        self.group1 = nn.Sequential(
            nn.Conv3d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(1, 2, 2)))
        self.group2 = nn.Sequential(
            nn.Conv3d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm3d(128),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2)))
        self.group3 = nn.Sequential(
            nn.Conv3d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm3d(256),
            nn.ReLU(),
            nn.Conv3d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm3d(256),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2)))
        self.group4 = nn.Sequential(
            nn.Conv3d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm3d(512),
            nn.ReLU(),
            nn.Conv3d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm3d(512),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2)))
        self.group5 = nn.Sequential(
            nn.Conv3d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm3d(512),
            nn.ReLU(),
            nn.Conv3d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm3d(512),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(2, 2, 2), padding=(0, 1, 1)))

        last_duration = int(math.floor(sample_duration / 16))
        last_size = int(math.ceil(sample_size / 32))
        self.fc1 = nn.Sequential(
            nn.Linear((512 * last_duration * last_size * last_size) , 4096),
            nn.ReLU(),
            nn.Dropout(0.5))
        self.fc2 = nn.Sequential(
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5))
        self.fc = nn.Sequential(
            nn.Linear(4096, num_classes))         

        

    def forward(self, x):
        out = self.group1(x)
        out = self.group2(out)
        out = self.group3(out)
        out = self.group4(out)
        out = self.group5(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc(out)
        return out


def get_fine_tuning_parameters(model, ft_portion):
    if ft_portion == "complete":
        return model.parameters()

    elif ft_portion == "last_layer":
        ft_module_names = []
        ft_module_names.append('fc')

        parameters = []
        for k, v in model.named_parameters():
            for ft_module in ft_module_names:
                if ft_module in k:
                    parameters.append({'params': v})
                    break
            else:
                parameters.append({'params': v, 'lr': 0.0})
        return parameters

    else:
        raise ValueError("Unsupported ft_portion: 'complete' or 'last_layer' expected")


def get_model(**kwargs):
    """
    Returns the model.
    """
    model = C3D(**kwargs)
    return model


if __name__ == '__main__':
    model = get_model(sample_size = 112, sample_duration = 16, num_classes=600)
    model = model.cuda()
    model = nn.DataParallel(model, device_ids=None)
    print(model)

    input_var = Variable(torch.randn(8, 3, 16, 112, 112))
    output = model(input_var)
    print(output.shape)

In [ ]:
try:
  model
except NameError:
  import pretrainedmodels.utils as utils
  model = torch.load('drive/MyDrive/imagenet_nasnetalarge.pth')
  laod_img = utils.LoadImage()
  tf_img = utils.LoadImage()

img_file = './drive/MyDrive/002.jpeg'

input_img = load_img(img_file)
input_tensor = tf_img(input_img)
input_tensor = input_tensor.unsqueeze(0)
input = torch.autograd.Variable(input_tensor, requires_grad=False)

output_logits = model(input)

print("{} is [{}: {}]".format(img_file, output_logits.argmax(),
                              imagenet_class[int(output_logits.argmax())]))